# Exercice 3

## Résolution d'EDP par réseau de neurones dans le cadre HJB

Nous reprenons l'algorithme codé par les auteurs dans le cadre Allen-Cahn (partie 4.2 de l'article) pour l'adapter au cadre HJB (partie 4.3 de l'article).

L'algorithme de l'article s'intéresse à la résolution de l'équation différentielle :

$$\frac{\partial_u}{\partial_t}(t,x)+ \frac{1}{2}Trace\Big(\sigma(t,x)[\sigma(t,x)]*(Hess_x u)(t,x)\Big) + <\mu(t,x),(\nabla_x u)(t,x)> +f\Big(t,x,u(t,x),[(\nabla_x u)(t,x)]*\sigma (t,x)\Big) = 0$$

En particulier, dans le cadre HJB :
$$f(t,x,y,z) = -\vert \vert z \vert \vert ^2$$

$$g(x) = ln(\frac{1}{2}[1+\vert \vert x \vert \vert ^2] )$$

$$\sigma(t,x)w = \sqrt{2}w$$

$$\mu (t,x) = 0$$

La solution de l'EDP est alors  :

$$\frac{\partial_u}{\partial_t}(t,x)+ (\Delta_x u)(t,x) = \vert \vert (\nabla_x u)(t,x)\vert \vert ^2$$

In [ ]:
import time
import math
import tensorflow as tf
import numpy as np
from tensorflow.python.training.moving_averages import assign_moving_average
from scipy.stats import multivariate_normal as normal
from tensorflow.python.ops import control_flow_ops
from tensorflow import random_normal_initializer as norm_init
from tensorflow import random_uniform_initializer as unif_init
from tensorflow import constant_initializer as const_init

class SolveHJB(object):
    def __init__(self, sess):
     self.sess = sess
     self.d = 100
     self.T = 1
     self.n_time = 20
     self.n_layer =4
     self.n_neuron = [self.d, self.d+10, self.d+10, self.d]
     self.batch_size = 64
     self.valid_size = 256
     self.n_maxstep = 2000
     self.n_displaystep = 10
     self.learning_rate = 1/100
     self.Yini = [0.3, 0.6]
     self.h = (self.T+0.0)/self.n_time
     self.sqrth = math.sqrt(self.h)
     self.t_stamp = np.arange(0, self.n_time)*self.h
     self._extra_train_ops = []
     self._lambda = 1.0
     
    def train(self):
     start_time = time.time()
     self.global_step = tf.get_variable('global_step', [],initializer=tf.constant_initializer(1),trainable=False , dtype=tf.int32)
     trainable_vars = tf.trainable_variables()
     grads = tf.gradients(self.loss, trainable_vars)
     optimizer = tf.train.AdamOptimizer(self.learning_rate)
     apply_op = optimizer.apply_gradients(zip(grads, trainable_vars),global_step=self.global_step)
     train_ops = [apply_op] + self._extra_train_ops
     self.train_op = tf.group(*train_ops)
     self.loss_history =[]
     self.init_history =[]
     dW_valid , X_valid= self.sample_path(self.valid_size) 
     feed_dict_valid ={self.dW: dW_valid , self.X: X_valid , self.is_training: False}
     step=1
     self.sess.run(tf.global_variables_initializer())
     temp_loss = self.sess.run(self.loss, feed_dict=feed_dict_valid)
     temp_init = self.Y0.eval()[0]
     self.loss_history.append(temp_loss)
     self.init_history.append(temp_init)
     print("step: %5u, loss: %.4e, "%(0, temp_loss)+"Y0: %.4e,runtime: %4u"%(temp_init,time.time()-start_time+self.t_bd))
     for i in range(self.n_maxstep+1):
         step = self.sess.run(self.global_step)
         dW_train , X_train = self.sample_path(self.batch_size)
         self.sess.run(self.train_op,feed_dict={self.dW: dW_train, self.X: X_train,self.is_training: True})
         if  step % self.n_displaystep==0:
             temp_loss = self.sess.run(self.loss,feed_dict=feed_dict_valid)
             temp_init = self.Y0.eval()[0]
             self.loss_history.append(temp_loss)
             self.init_history.append(temp_init)
             print("step:%5u,loss:%.4e, " %(step,temp_loss), "Y0:% .4e,runtime:% 4u" %(temp_init,time.time()-start_time+self.t_bd))
         step += 1
     end_time = time.time()
     print("running time:%.3f s"%(end_time -start_time+self.t_bd))
             
    def build(self):
                 start_time = time.time()
                 self.dW =tf.placeholder(tf.float64 ,[None, self.d, self.n_time], name='dW')
                 self.X = tf.placeholder(tf.float64 ,[None, self.d, self.n_time+1],name='X')
                 self.is_training = tf.placeholder(tf.bool)
                 self.Y0 = tf.Variable(tf.random_uniform([1],minval=self.Yini[0],maxval=self.Yini[1],dtype=tf.float64));
                 self.Z0 = tf.Variable(tf.random_uniform([1, self.d],minval=-.1,maxval=.1,dtype=tf.float64))
                 self.allones = tf.ones(shape=[tf.shape(self.dW)[0], 1],dtype=tf.float64)
                 Y= self.allones * self.Y0
                 Z= tf.matmul(self.allones , self.Z0)
                 with tf.variable_scope('forward', reuse=tf.AUTO_REUSE):
                     for t in range(0,self.n_time-1):
                         Y= Y - self.f_tf(self.t_stamp[t],self.X[:, :, t], Y, Z)*self.h
                         Y= Y + tf.reduce_sum(Z*self.dW[:, :, t], 1,keep_dims=True)
                         Z = self._one_time_net(self.X[:, :, t+1],str(t+1))/self.d
                     Y = Y - self.f_tf(self.t_stamp[self.n_time -1],self.X[:, :, self.n_time -1],Y, Z)*self.h
                     Y = Y + tf.reduce_sum(Z*self.dW[:, :, self.n_time-1], 1,keep_dims=True)
                     term_delta = Y - self.g_tf(self.T,self.X[:, :, self.n_time])
                     self.clipped_delta = tf.clip_by_value(term_delta , -50.0, 50.0)
                     self.loss = tf.reduce_mean(self.clipped_delta**2)
                     self.t_bd = time.time()-start_time
                
    def sample_path(self, n_sample):
        dW_sample = np.zeros([n_sample , self.d, self.n_time])
        X_sample = np.zeros([n_sample, self.d, self.n_time+1])
        for i in range(self.n_time):
            dW_sample[:, :, i] = np.reshape(normal.rvs(mean=np.zeros(self.d),cov=1,size=n_sample)*self.sqrth,(n_sample, self.d))
            X_sample[:, :, i+1] = X_sample[:, :, i] + np.sqrt(2) * dW_sample[:, :, i]
        return dW_sample , X_sample
        
   # def f_tf(self , t, X,  Y, Z):
   #    return Y-tf.pow(Y, 3)
   # def g_tf(self, t, X):
   #    return 0.5/(1 + 0.2*tf.reduce_sum(X**2, 1, keep_dims=True))
    
    def f_tf(self, t, x, y, z):
        return -self._lambda * tf.reduce_sum(tf.square(z), 1, keep_dims=True)

    def g_tf(self, t, x):
        return tf.log((1 + tf.reduce_sum(tf.square(x), 1, keep_dims=True)) / 2)
   
    
    def _one_time_net(self, x, name):
       with tf.variable_scope(name):
           x_norm = self._batch_norm(x, name='layer0_normal')
           layer1 = self._one_layer(x_norm, self.n_neuron[1],name='layer1')
           layer2 = self._one_layer(layer1, self.n_neuron[2],name='layer2')
           z = self._one_layer(layer2, self.n_neuron[3], activation_fn=None, name='final')
       return z
       
        
    def _one_layer(self, input_, out_sz,activation_fn=tf.nn.relu,std=5.0, name='linear'):
            with tf.variable_scope(name):
                shape = input_.get_shape().as_list()
                w = tf.get_variable('Matrix',[shape[1], out_sz],tf.float64 , norm_init(stddev= std/np.sqrt(shape[1]+out_sz))) 
                hidden=tf.matmul(input_ , w)
                hidden_bn = self._batch_norm(hidden, name='normal')
            if activation_fn != None:
                    return activation_fn(hidden_bn)
            else:
                    return hidden_bn
                
    def _batch_norm(self, x, name):
            with tf.variable_scope(name):
                params_shape = [x.get_shape()[-1]]
                beta = tf.get_variable('beta', params_shape , tf.float64 ,norm_init(0.0, stddev=0.1,dtype=tf.float64))
                gamma =tf.get_variable('gamma', params_shape , tf.float64 ,unif_init(0.1, 0.5, dtype=tf.float64))
                mv_mean = tf.get_variable('moving_mean', params_shape,tf.float64,const_init(0.0, tf.float64), trainable=False)
                mv_var = tf.get_variable('moving_variance', params_shape,tf.float64,const_init(1.0, tf.float64), trainable=False)
                mean, variance = tf.nn.moments(x, [0], name='moments')
                self._extra_train_ops.append(assign_moving_average(mv_mean, mean, 0.99))
                self._extra_train_ops.append(assign_moving_average(mv_var , variance , 0.99))
                mean , variance = control_flow_ops.cond(self.is_training ,lambda: (mean, variance),lambda: (mv_mean , mv_var))
                y = tf.nn.batch_normalization(x, mean, variance, beta, gamma, 1e-6)
                y.set_shape(x.get_shape())
                return y
        
def hjb_training(filename="./HJB_d10.csv", display_step=10):
       tf.reset_default_graph()
       with tf.Session() as sess:
           tf.set_random_seed(1)
           print ("Begin to solve HJB")
           model = SolveHJB(sess)
           model.build()
           #for m in range(1, 2000,200):
           #print ("m = " + str(m))
           model.n_displaystep = display_step
           model.train()
           output = np.zeros((len(model.init_history), 3))
           output[:, 1] = model.loss_history 
                      
           np.savetxt(filename, output ,fmt=['%d', '%.5e', '%.5e'], delimiter=",",header="step, loss function, " +  "target value ,runtime", comments='')
           return output
if __name__ == '__main__':
               np.random.seed(1) 
               hjb_training()


C:\Programs\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Begin to solve HJB
Instructions for updating:
keep_dims is deprecated, use keepdims instead
step:     0, loss: 1.7901e+01, Y0: 3.5068e-01,runtime:   31
step:   10,loss:1.6999e+01,  Y0: 4.5058e-01,runtime:  43
step:   20,loss:1.6028e+01,  Y0: 5.4995e-01,runtime:  44
step:   30,loss:1.4994e+01,  Y0: 6.4841e-01,runtime:  46
step:   40,loss:1.3862e+01,  Y0: 7.4537e-01,runtime:  47
step:   50,loss:1.2632e+01,  Y0: 8.4033e-01,runtime:  48
step:   60,loss:1.1238e+01,  Y0: 9.3259e-01,runtime:  50
step:   70,loss:9.7112e+00,  Y0: 1.0218e+00,runtime:  51
step:   80,loss:8.1476e+00,  Y0: 1.1078e+00,runtime:  52
step:   90,loss:6.5952e+00,  Y0: 1.1883e+00,runtime:  53
step:  100,loss:5.1858e+00,  Y0: 1.2623e+00,runtime:  54
step:  110,loss:4.0413e+00,  Y0: 1.3280e+00,runtime:  55
step:  120,loss:3.2671e+00,  Y0: 1.3833e+00,runtime:  57
step:  130,loss:2.8424e+00,  Y0: 1.4282e+00,runtime:  58
step:  140,loss:2.6664e+00,  Y0: 1.4635e+00,runtime:  59
step:  150,loss:2.6083e+00,  Y0: 1.4922e+00,runtim

step: 1390,loss:2.2232e-02,  Y0: 4.5847e+00,runtime: 243
step: 1400,loss:2.2103e-02,  Y0: 4.5866e+00,runtime: 245
step: 1410,loss:2.2144e-02,  Y0: 4.5877e+00,runtime: 246
step: 1420,loss:2.2055e-02,  Y0: 4.5881e+00,runtime: 248
step: 1430,loss:2.1980e-02,  Y0: 4.5884e+00,runtime: 250
step: 1440,loss:2.1805e-02,  Y0: 4.5897e+00,runtime: 252
step: 1450,loss:2.1643e-02,  Y0: 4.5915e+00,runtime: 253
step: 1460,loss:2.1548e-02,  Y0: 4.5930e+00,runtime: 255
step: 1470,loss:2.1549e-02,  Y0: 4.5939e+00,runtime: 257
step: 1480,loss:2.1457e-02,  Y0: 4.5945e+00,runtime: 259
step: 1490,loss:2.1257e-02,  Y0: 4.5956e+00,runtime: 260
step: 1500,loss:2.1147e-02,  Y0: 4.5960e+00,runtime: 262
step: 1510,loss:2.0944e-02,  Y0: 4.5958e+00,runtime: 264
step: 1520,loss:2.0907e-02,  Y0: 4.5967e+00,runtime: 265
step: 1530,loss:2.0919e-02,  Y0: 4.5981e+00,runtime: 267
step: 1540,loss:2.1063e-02,  Y0: 4.5985e+00,runtime: 269
step: 1550,loss:2.1211e-02,  Y0: 4.5983e+00,runtime: 271
step: 1560,loss:2.1331e-02,  Y0

In [ ]:
hjb_training("./HJB_d100.csv", 100)

Begin to solve HJB
step:     0, loss: 1.7803e+01, Y0: 3.5068e-01,runtime:   38
step:  100,loss:4.9219e+00,  Y0: 1.2538e+00,runtime:  62
step:  200,loss:2.4632e+00,  Y0: 1.5661e+00,runtime:  74
step:  300,loss:2.2577e+00,  Y0: 1.7798e+00,runtime:  89
step:  400,loss:2.0437e+00,  Y0: 2.0286e+00,runtime: 106
step:  500,loss:1.8501e+00,  Y0: 2.3142e+00,runtime: 124


Nous traçons la fonction de perte avec 200 époques de 10 itérations. On remarque que l'algorithme diminue rapidement la perte avant d'atteindre un palier symbole de stagnation de l'apprentissage. Nous traçons également le graphique basé sur 20 époques de 100 itérations. 

Nous observons qu'avec un nombre d'itérations moins important dans le minibatch, la perte diminue plus vite. 
Par contre le palier de stagnation est atteint à peu près au même moment dans les deux graphiques.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
loss_history = pd.read_csv("HJB_d10.csv")
plt.figure(figsize=(10,5))
plt.subplot(121)
plt.plot(loss_history.iloc[1:,1].values)
plt.xlabel('Number of iterations')
plt.ylabel('Loss')

loss_history2 = pd.read_csv("HJB_d100.csv")
plt.subplot(122)
plt.plot(loss_history2.iloc[1:,1].values)
plt.xlabel('Number of iterations')
plt.ylabel('Loss')
plt.show()